# Compartimental Model Simulator

**Created by:** Samuel Ropert  
**Creation date:** 04/08/2020  
**Institution:** Computational Biology Lab - Fundación Ciencia y Vida, Chile  

## SEIR
This jupyter notebook shows how to build a single SEIR simulation object with different quarantine scenarios. In this example we show 4 quarantine scenarios, 2 with total quarantines, and 2 with dynamic quarantines.


## Import Libraries

In [1]:
import sys
from pathlib import Path
sys.path.insert(1, '../src/SEIR/')
sys.path.insert(1, '../src/utils/')

from datetime import datetime
import numpy as np
from numpy import linalg as LA
import pandas as pd
from time import time

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
else:
    print('OS not detected :-|')


from class_SEIR import SEIR
from importdata import ImportData as importdata
from Quarantine import Quarantine

Linux


## Epidemiological Parameters
* **beta:** Infection rate
* **mu:** Initial exposed obtained from the initial infected mu=E0/I0
* **Scale Factor:** Proportion of real infected compared to reported ones (1: all the infecteds are reported)
* **Sero Prevalence Factor:** Adjust the proportion of the population that enters the virus dynamics
* **Exposed Infection:** rate compared to the infected (0 the don't infect, 1 the infect in the same rate as the infected )

## Simulation Parameters

In [9]:
beta = 0.2 # Contagion rate
mu = 1.5 # E0/I0 initial rate
SeroPrevalence = 1
expinfection = 0

# Simulation time
tsim = 500
# Population
population = 1000000
# Initial Active Infected 
I0 = 100

I_ac0 = 100
# Kinetic Saturation: 0 for mass action mixing
k=0

### Quarantines

Quarantine object constructor:
 
 
 ```Quarantine(rem_mov,max_mov=0.85,qp=0,iqt=0,fqt=1000,movfunct = 'once')```
 
 * rem_mov: Remanent mobility during Quarantine
 * max_mov: Mobility during non quarantine periods
 * qp: Quarantine period (for dynamic quarantines)
 * iqt: Initial quarantine time
 * fqt: Final quarantine time
 * movfunct: Mobility function 

Mobility function types:
 * once: Total quarantine between iqt and fqt
 * square: Periodic quaratine with qp period

In [10]:
rem_mob = 0.4 # Mobility during quarantine
max_mob = 0.8 # Maximum mobility
# Total quarantine

s1 = Quarantine(0.5)
s2 = Quarantine(0.5,0.8,qp=14,iqt=50,fqt = 500,movfunct='square')

quarantines = [s1.alpha,s2.alpha]

In [13]:
# Plot quarantine vs time
s2.plot()

## Create simulation Object

In [16]:
simulation = SEIR(tsim=tsim,alpha=s2.alpha,beta=beta,mu=mu,k=k,I=I0,population=population,expinfection=1,SeroPrevFactor=1)

## Simulate
The different scenarios are simulated in parallel threads 

In [17]:
simulation.integr_sci(0,tsim,0.01)

  message: 'The solver successfully reached the end of the integration interval.'
     nfev: 3156
     njev: 172
      nlu: 172
      sol: None
   status: 0
  success: True
        t: array([0.00000000e+00, 1.12938488e-06, 2.25876976e-06, 5.33060911e-03,
       1.06589595e-02, 1.59873098e-02, 6.92708133e-02, 1.22554317e-01,
       1.75837820e-01, 2.29121324e-01, 4.88676528e-01, 7.48231733e-01,
       1.00778694e+00, 1.26734214e+00, 1.52689735e+00, 1.97643515e+00,
       2.42597296e+00, 2.87551077e+00, 3.32504857e+00, 3.77458638e+00,
       4.32390568e+00, 4.87322498e+00, 5.42254428e+00, 5.97186358e+00,
       6.52515027e+00, 7.07843696e+00, 7.63172365e+00, 7.98117412e+00,
       8.33062459e+00, 8.68007505e+00, 9.43550793e+00, 1.01909408e+01,
       1.09463737e+01, 1.36845352e+01, 1.64226967e+01, 1.71072371e+01,
       1.77917775e+01, 1.84763179e+01, 1.98453987e+01, 2.12144794e+01,
       2.25835602e+01, 2.55430031e+01, 2.85024460e+01, 3.14618889e+01,
       3.44213318e+01, 3.95265265e+

# Simulation Analysis

## Plots
This libraries have predefined plot functions which plot the main epidemiological variables.
Each function has the following optional arguments:
* days [int] Number of days to display
* showparams [bool] Display simulation parameters 
* ylim [int] Limit the vertical axis
* norm [int/float] Normalize the results


In [18]:
plt.plot(simulation.t,simulation.S,label='S',color = 'blue')
plt.plot(simulation.t,simulation.E,label='E',color = 'cyan')
plt.plot(simulation.t,simulation.I,label='I',color = 'red')
plt.plot(simulation.t,simulation.R,label='R',color = 'green')
plt.legend(loc=0)
plt.show()

In [ ]:
# Epidemiological curves
simulation.plotseir(days = 300)

In [ ]:
# Active Infected curves
simulation.plotActiveInfected(showparams=True,days = 650)

In [ ]:
# Accumulated infected
simulation.plotAccumulatedInfected()

In [ ]:
# Daily infected
simulation.plotDailyInfected(days=400)

In [ ]:
# Exposed
simulation.plotExposed()

In [ ]:
# Exposed
simulation.plotQuarantines(days=100)

### Variables:
The simulation object contains several methods and variables with its results

In [ ]:
# Susceptibles
simulation.S
# Exposed
simulation.E
# Infected
simulation.I
# Recovered
simulation.R


In [ ]:
# Peak Values per each scenario
simulation.peak

In [ ]:
# Peak time
simulation.peak_t

In [ ]:
simulation.showscenarios()